In [ ]:
import numpy as np
import pandas as pd

import tkinter as tk
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

import os

import mne
from mne import io
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from mne.datasets import sample
from mne.stats import permutation_t_test
import scipy.stats as stats
from scipy.stats import shapiro

import sklearn.preprocessing as preprocessing

In [ ]:
#Preprocessing Functions

def preprocess_data(file):

            #convert csv to data frame
            data = np.genfromtxt(file, delimiter=',')
            df = pd.DataFrame(data)

            #select only raw data columns
            subset = df.iloc[:, 21:25]
            subset.dropna(inplace=True)
            ch_names = ['TP9', 'AF7', 'AF8', 'TP10']

            #need to transppose for the mne function to work
            subsetT = subset.transpose()
            sfreq = 256  # Sampling frequency

            #create mne recognizeable object
            info = mne.create_info(ch_names=ch_names, sfreq=sfreq)
            raw = mne.io.RawArray(subsetT, info)

            #setting raw columns to eeg channel type
            for i in ch_names:
                raw.set_channel_types({i: 'eeg'})

            #setting basic reference scalp electrode channel positions and filtering out less than 1 hz and greater than 55 hz
            raw.set_montage('standard_1020')
            Filt_raw = raw.filter(l_freq=1., h_freq=55., fir_design='firwin')

            return Filt_raw


In [ ]:
#Setting up a dictionary of all the files in the directory to reference

Preprocessed = dict()

#looping through all the files in the directory and preprocessing them

for i in os.listdir('/Users/danielbarrowsmacbook/Desktop/Lab Reports/Final Project/Data_for_processing'):
    if i.endswith('.csv'):
        preprocess_data(i)
        Preprocessed[i] = preprocess_data(i)

In [ ]:
# Power Spectrum Density calculations for each individual in each condition

fmin, fmax = 3, 55 # look at frequencies between 4 and 55Hz

for k in Preprocessed:
    if k.__contains__('Subject1_Control'):
        concatenated_sub1_contr = mne.concatenate_raws([Preprocessed[k]])
    if k.__contains__('Subject1_Reward'):
        concatenated_sub1_reward = mne.concatenate_raws([Preprocessed[k]])
    if k.__contains__('Subject1_Punishment'):
        concatenated_sub1_punish = mne.concatenate_raws([Preprocessed[k]])
fig, (ax1, ax2, ax3) = plt.subplots(3)
concatenated_sub1_contr.plot_psd(ax=ax1, fmin=fmin, fmax=fmax, average=True)
concatenated_sub1_reward.plot_psd(ax=ax2, fmin=fmin, fmax=fmax, average=True)
concatenated_sub1_punish.plot_psd(ax=ax3, fmin=fmin, fmax=fmax, average=True)
ax1.set_title('Subject 1 Control Power Density Spectrum')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Power (dB)')
ax2.set_title('Subject 1 Reward Power Density Spectrum')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Power (dB)')
ax3.set_title('Subject 1 Punishment Power Density Spectrum')
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Power (dB)')
fig.tight_layout()


for k in Preprocessed:
    if k.__contains__('Subject2_Control'):
        concatenated_sub2_contr = mne.concatenate_raws([Preprocessed[k]])
    if k.__contains__('Subject2_Reward'):
        concatenated_sub2_reward = mne.concatenate_raws([Preprocessed[k]])
    if k.__contains__('Subject2_Punishment'):
        concatenated_sub2_punish = mne.concatenate_raws([Preprocessed[k]])
fig, (ax1, ax2, ax3) = plt.subplots(3)
concatenated_sub2_contr.plot_psd(ax=ax1, fmin=fmin, fmax=fmax, average=True)
concatenated_sub2_reward.plot_psd(ax=ax2, fmin=fmin, fmax=fmax, average=True)
concatenated_sub2_punish.plot_psd(ax=ax3, fmin=fmin, fmax=fmax, average=True)
ax1.set_title('Subject 2 Control Power Density Spectrum')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Power (dB)')
ax2.set_title('Subject 2 Reward Power Density Spectrum')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Power (dB)')
ax3.set_title('Subject 2 Punishment Power Density Spectrum')
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Power (dB)')
fig.tight_layout()

for k in Preprocessed:
    if k.__contains__('Subject3_Control'):
        concatenated_sub3_contr = mne.concatenate_raws([Preprocessed[k]])
    if k.__contains__('Subject3_Reward'):
        concatenated_sub3_reward = mne.concatenate_raws([Preprocessed[k]])
    if k.__contains__('Subject3_Punishment'):
        concatenated_sub3_punish = mne.concatenate_raws([Preprocessed[k]])
fig, (ax1, ax2, ax3) = plt.subplots(3)
concatenated_sub3_contr.plot_psd(ax=ax1, fmin=fmin, fmax=fmax, average=True)
concatenated_sub3_reward.plot_psd(ax=ax2, fmin=fmin, fmax=fmax, average=True)
concatenated_sub3_punish.plot_psd(ax=ax3, fmin=fmin, fmax=fmax, average=True)
ax1.set_title('Subject 3 Control Power Density Spectrum')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Power (dB)')
ax2.set_title('Subject 3 Reward Power Density Spectrum')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Power (dB)')
ax3.set_title('Subject 3 Punishment Power Density Spectrum')
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Power (dB)')
fig.tight_layout()

In [ ]:
#Stats Comparing conditions

#Experimental Group Concatenation from 3 subjects and normalization

for k in Preprocessed:
    if k.__contains__('Control'):
        all_controls = mne.concatenate_raws([Preprocessed[k]])
        all_cont_scale = preprocessing.scale(all_controls.get_data())


for k in Preprocessed:
    if k.__contains__('Reward'):
        all_rewards = mne.concatenate_raws([Preprocessed[k]])
        all_rew_scale = preprocessing.scale(all_rewards.get_data())

for k in Preprocessed:
    if k.__contains__('Punishment'):
        all_punishments = mne.concatenate_raws([Preprocessed[k]])
        all_pun_scale = preprocessing.scale(all_punishments.get_data())


In [ ]:
#Make sure all arrays are the same length

cut_controls = all_cont_scale[:, 0:8190]
cut_rewards = all_rew_scale[:, 0:8190]
cut_punishments = all_pun_scale[:, 0:8190]

In [ ]:
#T-Test for Control vs Reward, Control vs Punishment, Reward vs Punishment
# For the two channels of interest, AF7 and AF8


AF7_contr_rew = stats.ttest_ind(cut_controls[1], cut_rewards[1])
Af7_contr_pun = stats.ttest_ind(cut_controls[1], cut_punishments[1])
AF7_rew_pun = stats.ttest_ind(cut_rewards[1], cut_punishments[1])

AF8_contr_rew = stats.ttest_ind(cut_controls[2], cut_rewards[2])
Af8_contr_pun = stats.ttest_ind(cut_controls[2], cut_punishments[2])
AF8_rew_pun = stats.ttest_ind(cut_rewards[2], cut_punishments[2])


print("AF7 Channel Controls vs. Rewards: ", AF7_contr_rew)
print("AF7 Controls vs. Punishments: ", Af7_contr_pun)
print("AF7 Rewards vs. Punishments: ", AF7_rew_pun)

print("AF8 Channel Controls vs. Rewards: ", AF8_contr_rew)
print("AF8 Controls vs. Punishments: ", Af8_contr_pun)
print("AF8 Rewards vs. Punishments: ", AF8_rew_pun)

In [ ]:
# ica = mne.preprocessing.ICA(n_components=4, random_state=97, )
# ica.fit(Filt_raw)
# ica.plot_components()